In [27]:
!pip install openai langchain==0.1.14
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 7.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.36
    Uninstalling langchain-core-0.2.36:
      Successfully uninstalled langchain-core-0.2.36
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.23 requires langchain-core<0.3.0,>=0.2.35, but you have langchain-core 0.1.52 which is incompatible.
langchain-milvus 0.1.4 requires langchain-core<0.3.0,>=0.2.20, but you have langchain-core 0.1.52 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 7.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.9-cp39-cp39-li

In [33]:
import os
import psycopg2
from psycopg2.extras import DictCursor
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.evaluation import load_evaluator

In [39]:
TOP_P = 0.95
NUM_PRED = 512
MODEL = "mistral"
TEMPERATURE = 0.01
REPETITION_PENALTY = 1.03
INFERENCE_SERVER_URL = "http://llm.ic-shared-llm.svc.cluster.local:11434"

In [35]:
def infer_with_template(input_text, template):
    llm = Ollama(
        base_url=INFERENCE_SERVER_URL,
        model=MODEL,
        top_p=TOP_P,
        temperature=TEMPERATURE,
        num_predict=NUM_PRED,
        repeat_penalty=REPETITION_PENALTY,
        callbacks=[StreamingStdOutCallbackHandler()]
    )

    PROMPT = PromptTemplate.from_template(template)
    llm_chain = LLMChain(llm=llm, prompt=PROMPT)
    return llm_chain.run(input_text)

In [ ]:
conn_params = {
    "dbname": os.getenv('DATABASE_NAME'),
    "user": os.getenv('DATABASE_USER'),
    "password": os.getenv('DATABASE_PASS'),
    "host": os.getenv('DATABASE_HOST'),
    "port": os.getenv('DATABASE_PORT')
}

In [ ]:
incident_id = 1
entity_name = os.getenv('ENTITY_TO_PROCES')

In [ ]:
select_record_by_id_query = """
SELECT * 
FROM fracture_incidents
WHERE id = %s;
"""

update_record_query = f"""
UPDATE fracture_incidents
SET 
    {entity_name} = %s
WHERE 
    id = %s;
"""

In [52]:
record = {}

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor(cursor_factory=DictCursor)

    cursor.execute(select_record_by_id_query, (incident_id,))
    record = cursor.fetchone()
    
    if record:
        print("Retrieved record:")
        print(record)
    else:
        print(f"No record found with ID {incident_id}.")

except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
finally:
    if conn:
        cursor.close()
        conn.close()
        print("Database connection closed.")

Retrieved record:
[1, '24 januari 2024, 19:00', None, 'In Amsterdam heeft een fietser op 24 januari 2024 rond 19:00 een ernstig ongeluk gehad terwijl hij door een drukke straat reed. De fietser raakte een gladde plek op de weg en kwam ongelukkig ten val. Na de val klaagde de fietser over hevige pijn in de enkel, die direct begon op te zwellen. Omstanders hielpen de fietser overeind en alarmeerden een ambulance. In het ziekenhuis werd een röntgenfoto gemaakt, en de arts vermoedt een breuk in de enkel. Het slachtoffer is opgenomen voor verdere observatie en behandeling.', 'accident_0001', None, 'unprocessed']
Database connection closed.


In [37]:
with open(f"templates/{entity_name}_template.txt") as f:
        template = f.read()

In [40]:
entity = infer_with_template(record['description'], template)

/opt/app-root/lib64/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


24 januari 2024, 19:00

In [51]:
update_data = (
    entity,
    incident_id
)

try:
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()

    cursor.execute(update_record_query, update_data)
    
    conn.commit()
    print(f"Record with ID {incident_id} updated successfully.")

except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    if conn:
        cursor.close()
        conn.close()
        print("Database connection closed.")

Record with ID 1 updated successfully.
Database connection closed.
